In [1]:
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(7)

In [2]:
def load_csv():
    with open('../data/subject_grades.csv', mode='r') as csv_file:
        data = list(csv.reader(csv_file, delimiter=',', quotechar='"'))
        data.pop(0)
        
        data = [[float(y) for y in x] for x in data]
        
        train_data = data[:int(len(data)*80/100)]
        test_data = data[int(len(data)*80/100):]
        
        return train_data, test_data

In [3]:
class FinalGradeNN(nn.Module):
    def __init__(self):
        super(FinalGradeNN, self).__init__()
        
        self.fc1 = nn.Linear(3, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 64)
        self.output = nn.Linear(64, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.output(x)
        
        return torch.sigmoid(x)
    
net = FinalGradeNN()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
loss_func = torch.nn.BCELoss()

print(net)

FinalGradeNN(
  (fc1): Linear(in_features=3, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
)


In [4]:
train_data, test_data = load_csv()


y_train = torch.FloatTensor([i[-1:] for i in train_data])
x_train = torch.FloatTensor([i[:3] for i in train_data])

In [5]:
epochs = 500

for epoch in range(epochs):
    predict = net(x_train)
    
    loss = loss_func(predict, y_train)
    
    if (epoch+1) % 100 == 0:
        print(epoch+1, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

100 0.09662337601184845
200 0.06538963317871094
300 0.010885565541684628
400 0.0035752442199736834
500 0.0016389022348448634


In [6]:
i = 1
correct_predict = 0

for entry in test_data:
    expected = entry[-1:][0]
    predicted = net.forward(torch.FloatTensor(entry[:3])).item()
    rounded_predicted = round(predicted)
    
    if expected == rounded_predicted:
        correct_predict += 1
    
    print("{}. Expected value: {} - Predicted value: {} ({})"
           .format(i,
                   expected,
                   rounded_predicted,
                  predicted),
          'NOT EQUAL' if expected != rounded_predicted else '')
    i += 1
    
print('Correct predictions: ' + str(correct_predict) +
      ' - Incorrect: ' + str(len(test_data) - correct_predict))

1. Expected value: 0.0 - Predicted value: 0 (1.5241254961360408e-23) 
2. Expected value: 0.0 - Predicted value: 0 (2.0410489433558138e-13) 
3. Expected value: 0.0 - Predicted value: 0 (0.006660603452473879) 
4. Expected value: 0.0 - Predicted value: 0 (0.05127737298607826) 
5. Expected value: 1.0 - Predicted value: 1 (1.0) 
6. Expected value: 0.0 - Predicted value: 0 (2.410985051943984e-15) 
7. Expected value: 1.0 - Predicted value: 1 (1.0) 
8. Expected value: 0.0 - Predicted value: 0 (1.8315979062353424e-18) 
9. Expected value: 1.0 - Predicted value: 1 (1.0) 
10. Expected value: 1.0 - Predicted value: 1 (1.0) 
11. Expected value: 1.0 - Predicted value: 1 (0.9999617338180542) 
12. Expected value: 1.0 - Predicted value: 1 (1.0) 
13. Expected value: 0.0 - Predicted value: 0 (2.129332965045434e-17) 
14. Expected value: 0.0 - Predicted value: 0 (6.909776573138515e-08) 
15. Expected value: 1.0 - Predicted value: 1 (1.0) 
16. Expected value: 1.0 - Predicted value: 1 (0.9907529354095459) 
17.

260. Expected value: 1.0 - Predicted value: 1 (0.99801105260849) 
261. Expected value: 1.0 - Predicted value: 1 (0.9999991655349731) 
262. Expected value: 0.0 - Predicted value: 0 (1.987529856251058e-07) 
263. Expected value: 1.0 - Predicted value: 1 (0.9734916687011719) 
264. Expected value: 0.0 - Predicted value: 0 (0.0011126874014735222) 
265. Expected value: 1.0 - Predicted value: 1 (0.9999980926513672) 
266. Expected value: 0.0 - Predicted value: 0 (6.595942977583036e-05) 
267. Expected value: 1.0 - Predicted value: 1 (1.0) 
268. Expected value: 0.0 - Predicted value: 0 (0.0025688218884170055) 
269. Expected value: 1.0 - Predicted value: 1 (0.9826223850250244) 
270. Expected value: 0.0 - Predicted value: 0 (9.442349437449593e-06) 
271. Expected value: 0.0 - Predicted value: 0 (2.5689740012253255e-18) 
272. Expected value: 1.0 - Predicted value: 1 (1.0) 
273. Expected value: 1.0 - Predicted value: 1 (1.0) 
274. Expected value: 1.0 - Predicted value: 1 (0.9981180429458618) 
275. Exp

716. Expected value: 0.0 - Predicted value: 0 (9.647300855455125e-22) 
717. Expected value: 0.0 - Predicted value: 0 (1.5862551479818175e-38) 
718. Expected value: 0.0 - Predicted value: 0 (2.1364445768057294e-09) 
719. Expected value: 0.0 - Predicted value: 0 (2.509000558535114e-21) 
720. Expected value: 0.0 - Predicted value: 0 (1.4500684507312567e-11) 
721. Expected value: 0.0 - Predicted value: 0 (8.061289413774375e-09) 
722. Expected value: 1.0 - Predicted value: 1 (1.0) 
723. Expected value: 0.0 - Predicted value: 0 (2.8763807335963065e-07) 
724. Expected value: 1.0 - Predicted value: 1 (1.0) 
725. Expected value: 1.0 - Predicted value: 1 (1.0) 
726. Expected value: 0.0 - Predicted value: 0 (5.671744548174737e-37) 
727. Expected value: 0.0 - Predicted value: 0 (1.2403723525823063e-31) 
728. Expected value: 0.0 - Predicted value: 0 (0.01629362627863884) 
729. Expected value: 1.0 - Predicted value: 1 (0.9999988079071045) 
730. Expected value: 0.0 - Predicted value: 0 (2.06109223450

857. Expected value: 0.0 - Predicted value: 0 (0.02404521405696869) 
858. Expected value: 0.0 - Predicted value: 0 (5.4451858886750415e-05) 
859. Expected value: 0.0 - Predicted value: 0 (1.9635484406561718e-26) 
860. Expected value: 0.0 - Predicted value: 0 (5.006442499894989e-22) 
861. Expected value: 1.0 - Predicted value: 1 (1.0) 
862. Expected value: 0.0 - Predicted value: 0 (0.004687861073762178) 
863. Expected value: 1.0 - Predicted value: 1 (0.9999816417694092) 
864. Expected value: 0.0 - Predicted value: 0 (3.95126016883296e-06) 
865. Expected value: 0.0 - Predicted value: 0 (0.00952791515737772) 
866. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
867. Expected value: 0.0 - Predicted value: 0 (7.649671179119956e-18) 
868. Expected value: 0.0 - Predicted value: 0 (2.801323267195555e-27) 
869. Expected value: 0.0 - Predicted value: 0 (3.5766994463487747e-23) 
870. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
871. Expected value: 1.0 - Pre

1109. Expected value: 0.0 - Predicted value: 0 (5.294924868722929e-21) 
1110. Expected value: 1.0 - Predicted value: 1 (1.0) 
1111. Expected value: 1.0 - Predicted value: 1 (1.0) 
1112. Expected value: 0.0 - Predicted value: 0 (5.9383966191478025e-18) 
1113. Expected value: 1.0 - Predicted value: 1 (0.9999963045120239) 
1114. Expected value: 0.0 - Predicted value: 0 (8.283598010002183e-14) 
1115. Expected value: 1.0 - Predicted value: 1 (1.0) 
1116. Expected value: 1.0 - Predicted value: 1 (0.9999997615814209) 
1117. Expected value: 0.0 - Predicted value: 0 (2.3398484927383834e-07) 
1118. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
1119. Expected value: 1.0 - Predicted value: 1 (1.0) 
1120. Expected value: 0.0 - Predicted value: 0 (1.8386800328812347e-13) 
1121. Expected value: 1.0 - Predicted value: 1 (1.0) 
1122. Expected value: 0.0 - Predicted value: 0 (5.392603670734861e-22) 
1123. Expected value: 1.0 - Predicted value: 1 (1.0) 
1124. Expected value: 1.0 - Pre

...

19080. Expected value: 1.0 - Predicted value: 1 (0.9999290704727173) 
19081. Expected value: 0.0 - Predicted value: 0 (0.00010447979730088264) 
19082. Expected value: 1.0 - Predicted value: 1 (0.9999990463256836) 
19083. Expected value: 0.0 - Predicted value: 0 (1.4745317258326417e-14) 
19084. Expected value: 0.0 - Predicted value: 0 (5.018282339154523e-13) 
19085. Expected value: 0.0 - Predicted value: 0 (8.961314415414582e-30) 
19086. Expected value: 0.0 - Predicted value: 0 (2.106960442628284e-16) 
19087. Expected value: 1.0 - Predicted value: 1 (1.0) 
19088. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19089. Expected value: 1.0 - Predicted value: 1 (0.9998202919960022) 
19090. Expected value: 1.0 - Predicted value: 1 (1.0) 
19091. Expected value: 1.0 - Predicted value: 1 (0.9999459981918335) 
19092. Expected value: 0.0 - Predicted value: 0 (5.939058134174502e-09) 
19093. Expected value: 1.0 - Predicted value: 1 (1.0) 
19094. Expected value: 1.0 - Predicted val

19399. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19400. Expected value: 1.0 - Predicted value: 1 (0.9999984502792358) 
19401. Expected value: 1.0 - Predicted value: 1 (1.0) 
19402. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19403. Expected value: 1.0 - Predicted value: 1 (0.9997789263725281) 
19404. Expected value: 1.0 - Predicted value: 1 (0.9996023774147034) 
19405. Expected value: 1.0 - Predicted value: 1 (1.0) 
19406. Expected value: 0.0 - Predicted value: 0 (1.5722461527145626e-11) 
19407. Expected value: 0.0 - Predicted value: 0 (0.0376446358859539) 
19408. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19409. Expected value: 1.0 - Predicted value: 1 (0.982544481754303) 
19410. Expected value: 1.0 - Predicted value: 1 (0.9999204874038696) 
19411. Expected value: 1.0 - Predicted value: 1 (1.0) 
19412. Expected value: 0.0 - Predicted value: 0 (4.5046624416632656e-20) 
19413. Expected value: 0.0 - Predicted value: 0 (

19521. Expected value: 0.0 - Predicted value: 0 (3.0369524584811605e-16) 
19522. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19523. Expected value: 0.0 - Predicted value: 0 (2.089928057102952e-05) 
19524. Expected value: 0.0 - Predicted value: 0 (1.0107268288715222e-15) 
19525. Expected value: 1.0 - Predicted value: 1 (1.0) 
19526. Expected value: 0.0 - Predicted value: 0 (1.587003098380002e-11) 
19527. Expected value: 1.0 - Predicted value: 1 (1.0) 
19528. Expected value: 1.0 - Predicted value: 1 (1.0) 
19529. Expected value: 0.0 - Predicted value: 0 (1.9428088461888425e-23) 
19530. Expected value: 0.0 - Predicted value: 0 (9.479636991832283e-14) 
19531. Expected value: 0.0 - Predicted value: 0 (2.1007962036634353e-09) 
19532. Expected value: 0.0 - Predicted value: 0 (1.6862865548361583e-09) 
19533. Expected value: 1.0 - Predicted value: 1 (1.0) 
19534. Expected value: 0.0 - Predicted value: 0 (8.080349150880019e-16) 
19535. Expected value: 0.0 - Predicted value:

19769. Expected value: 1.0 - Predicted value: 1 (1.0) 
19770. Expected value: 1.0 - Predicted value: 1 (1.0) 
19771. Expected value: 0.0 - Predicted value: 0 (7.65435861131849e-21) 
19772. Expected value: 0.0 - Predicted value: 0 (1.920141841083023e-07) 
19773. Expected value: 0.0 - Predicted value: 0 (3.859173830278451e-06) 
19774. Expected value: 0.0 - Predicted value: 0 (6.877666127314951e-16) 
19775. Expected value: 0.0 - Predicted value: 0 (0.004050923511385918) 
19776. Expected value: 1.0 - Predicted value: 1 (0.9999966621398926) 
19777. Expected value: 1.0 - Predicted value: 1 (1.0) 
19778. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19779. Expected value: 1.0 - Predicted value: 1 (1.0) 
19780. Expected value: 0.0 - Predicted value: 0 (6.179463696298626e-08) 
19781. Expected value: 0.0 - Predicted value: 0 (1.9084756885945353e-08) 
19782. Expected value: 1.0 - Predicted value: 1 (0.9999995231628418) 
19783. Expected value: 1.0 - Predicted value: 1 (0.999999

19900. Expected value: 0.0 - Predicted value: 0 (1.0273350746492493e-11) 
19901. Expected value: 1.0 - Predicted value: 1 (1.0) 
19902. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19903. Expected value: 0.0 - Predicted value: 0 (2.6563186329361943e-13) 
19904. Expected value: 0.0 - Predicted value: 0 (1.2138067548353613e-27) 
19905. Expected value: 0.0 - Predicted value: 0 (1.7894423443021168e-10) 
19906. Expected value: 0.0 - Predicted value: 0 (1.1519967271778064e-11) 
19907. Expected value: 0.0 - Predicted value: 0 (2.9541581625380786e-07) 
19908. Expected value: 1.0 - Predicted value: 1 (1.0) 
19909. Expected value: 1.0 - Predicted value: 1 (1.0) 
19910. Expected value: 0.0 - Predicted value: 0 (4.629988126225726e-08) 
19911. Expected value: 0.0 - Predicted value: 0 (1.3257809428068867e-07) 
19912. Expected value: 1.0 - Predicted value: 1 (0.9734418392181396) 
19913. Expected value: 1.0 - Predicted value: 1 (1.0) 
19914. Expected value: 0.0 - Predicted value: 